In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import os
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
import spacy
nlp=spacy.load('en_core_web_sm')
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000
pd.set_option('display.max_colwidth', 200)  

In [25]:
data=pd.read_csv('/Users/mstudio/repository/viraltext/snippet-fasttext/two_extended_abbeville_banner_slave.csv')

In [2]:
os.chdir('/Users/mstudio/repository/viraltext/snippet-fasttext')

In [7]:
for i in tqdm(os.listdir()):
    if i.endswith('.csv'):
        data=lemmatization(pd.read_csv(i))
        data['5-gram']=data['lemma'].apply(lambda x: get_ngrams(x, 5))
        reuse=reuse_df(data)
        outname='reuse_'+i.lstrip('two_extended_').rstrip('.csv')+'.csv'
        outdir='/Users/mstudio/repository/viraltext/reuse/'
        if not os.path.exists(outdir):
            os.mkdir(outdir)
        fullname=os.path.join(outdir, outname)
        reuse.to_csv(fullname)

  0%|                                                     | 0/3 [00:00<?, ?it/s]/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_977/2480336597.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]','')
100%|███████████████████████████████████████| 3/3 [11:37:59<00:00, 13959.86s/it]


In [4]:
def lemmatization(dataframe:pd.DataFrame()):
    dataframe['stopword']=dataframe['context'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]','')
    dataframe['lower']=dataframe['punct'].str.lower()
    dataframe['lemma']=dataframe['lower'].apply(lambda row: ' '.join([w.lemma_ for w in nlp(row)]))
    dataframe['token']=dataframe['lemma'].apply(word_tokenize)
    return dataframe

In [27]:
%%time
data=lemmatization(data)

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_1754/2480336597.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]','')


CPU times: user 10.6 s, sys: 24.2 ms, total: 10.6 s
Wall time: 10.6 s


In [5]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n):
    entire=[]
    n_grams=ngrams(word_tokenize(text), n)
    for i in n_grams:
        entire.append(i)
    return entire

In [45]:
data.shape

(42742, 9)

In [ ]:
#data=data.head(5)

In [ ]:
get_ngrams('i need your love, i need your time', 3)

[('i', 'need', 'your'),
 ('need', 'your', 'love'),
 ('your', 'love', ','),
 ('love', ',', 'i'),
 (',', 'i', 'need'),
 ('i', 'need', 'your'),
 ('need', 'your', 'time')]

In [30]:
data['5-gram']=data['lemma'].apply(lambda x: get_ngrams(x, 5))

In [ ]:
data['5-gram'][0]

[('actually', 'perpetrate', 'south', 'institution', 'slavery'),
 ('perpetrate', 'south', 'institution', 'slavery', 'exclude'),
 ('south', 'institution', 'slavery', 'exclude', 'every'),
 ('institution', 'slavery', 'exclude', 'every', 'foot'),
 ('slavery', 'exclude', 'every', 'foot', 'territory'),
 ('exclude', 'every', 'foot', 'territory', 'acquire')]

In [ ]:
list(set(data['5-gram'][0]).intersection(data['5-gram'][0]))

[('institution', 'slavery', 'exclude', 'every', 'foot'),
 ('south', 'institution', 'slavery', 'exclude', 'every'),
 ('slavery', 'exclude', 'every', 'foot', 'territory'),
 ('exclude', 'every', 'foot', 'territory', 'acquire'),
 ('perpetrate', 'south', 'institution', 'slavery', 'exclude'),
 ('actually', 'perpetrate', 'south', 'institution', 'slavery')]

In [ ]:
test=[
('perpetrate', 'south', 'institution', 'slavery', 'exclude'),
('people', 'in', 'the', 'house', 'sleep')
]

In [ ]:
list(set(data['5-gram'][0]).intersection(test))

[('perpetrate', 'south', 'institution', 'slavery', 'exclude')]

In [ ]:
data['5-gram'][0]

[('actually', 'perpetrate', 'south', 'institution', 'slavery'),
 ('perpetrate', 'south', 'institution', 'slavery', 'exclude'),
 ('south', 'institution', 'slavery', 'exclude', 'every'),
 ('institution', 'slavery', 'exclude', 'every', 'foot'),
 ('slavery', 'exclude', 'every', 'foot', 'territory'),
 ('exclude', 'every', 'foot', 'territory', 'acquire')]

In [6]:
def reuse_df(data:pd.DataFrame()):
    no_reuse=[]
    pair_index1=[]
    pair_index2=[]
    pair_date1=[]
    pair_date2=[]
    pair_text1=[]
    pair_text2=[]
    for k in range(0,data.shape[0]):
        t=0
        for p in range(t,data.shape[0]):
            t+=1
            if t>k:
                if (k==p) & (len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))) == len(data['5-gram'][k])):
                    pass
                elif len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))) == 0:
                    pass
                elif len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))) >= 3:
                    no_reuse.append(len(list(set(data['5-gram'][k]).intersection(data['5-gram'][p]))))
                    pair_index1.append(k) 
                    pair_index2.append(p)
                    pair_date1.append(data.iloc[k]['date'])
                    pair_date2.append(data.iloc[p]['date'])
                    pair_text1.append(data.iloc[k]['lemma'])
                    pair_text2.append(data.iloc[p]['lemma'])
    return pd.DataFrame({'no_reuse':no_reuse, 'pair_index1':pair_index1, 'pair_index2':pair_index2, 
             'pair_date1':pair_date1, 'pair_date2':pair_date2, 
              'pair_text1':pair_text1, 'pair_text2':pair_text2
             })

In [44]:
pd.DataFrame({'no_reuse':no_reuse, 'pair_index1':pair_index1, 'pair_index2':pair_index2, 
             'pair_date1':pair_date1, 'pair_date2':pair_date2, 
              'pair_text1':pair_text1, 'pair_text2':pair_text2
             })

,no_reuse,pair_index1,pair_index2,pair_date1,pair_date2,pair_text1,pair_text2
0,3,15,136,1856-08-21-seq-4.txt,1856-09-18-seq-4.txt,penetrate it lie political liberal owe 110 slavish sorvilc allegiance party politician it advocate measure man,penetrate it political liberal owe slavish servile allegiance party politician it advocate measure man
1,3,15,283,1856-08-21-seq-4.txt,1856-05-08-seq-4.txt,penetrate it lie political liberal owe 110 slavish sorvilc allegiance party politician it advocate measure man,penetrate it political liberal owe slavish servile allegiance party politician it advocate measure man
2,3,15,618,1856-08-21-seq-4.txt,1856-06-05-seq-4.txt,penetrate it lie political liberal owe 110 slavish sorvilc allegiance party politician it advocate measure man,penetrate it political liberal owe slavish servile allegiance party politician it advocate measure man
3,3,15,723,1856-08-21-seq-4.txt,1856-02-28-seq-4.txt,penetrate it lie political liberal owe 110 slavish sorvilc allegiance party politician it advocate measure man,penetrate it politioul liberal owe uc slavish servile allegiance party politician it advocate measure man
4,3,15,1024,1856-08-21-seq-4.txt,1856-06-26-seq-4.txt,penetrate it lie political liberal owe 110 slavish sorvilc allegiance party politician it advocate measure man,ponetrate it political liherul owe slavish rervile allegiance party politician it advocate measure man
...,...,...,...,...,...,...,...
338,4,2552,2553,1858-03-25-seq-1.txt,1858-03-25-seq-1.txt,the law utah true recognize provide slavery hut slave lbefc jutuh technically slave,provide slavery hut slave lbefc jutuh technically slave owmwpitjj actually free community j
339,3,2608,2609,1856-02-14-seq-2.txt,1856-02-14-seq-2.txt,tion tlie cnsiirtin f friend 1 firslshould kansas tie make u slave c state we say location climate soil w,state we say location climate soil w production value f slave lahor tlie good master si iveall conspire h
340,7,2615,2616,1856-02-14-seq-2.txt,1856-02-14-seq-2.txt,dom lawrence t secondlycan kansas lie make slave h state thus far proslavery party,secondlycan kansas lie make slave h state thus far proslavery party triumph kmsa spile a ho
341,7,2660,2661,1858-04-29-seq-2.txt,1858-04-29-seq-2.txt,decision terrritory remain territorial condition slave territory amd thfat southern man may carry slave,territorial condition slave territory amd thfat southern man may carry slave there friend freedom emphatically
